In [ ]:
## 准备图片数据
train_data_dir = "E:/DeepLearning/PigRecog/TransferLearn/pigImages/train5"
validation_data_dir = "E:/DeepLearning/PigRecog/TransferLearn/pigImages/valid5"
img_width, img_height = 299,299

###
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(rescale=1./255,
        channel_shift_range=10,
        rotation_range=20,                           
        #shear_ range=0.2,                        
        zoom_range=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir , # 请自行修改文件所在目录
        target_size=(img_height, img_width),
        batch_size=16,
        class_mode='categorical',
        shuffle=False) 
#　验证集图像生成器
valid_generator= test_datagen .flow_from_directory(
        validation_data_dir,# 请自行修改文件所在目录
        target_size=(img_height, img_width),
        batch_size=16,
        class_mode='categorical',
        shuffle=False)


In [ ]:
### 
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D,Dropout
from keras import regularizers,initializers
from keras.models import Model
from keras.applications.inception_resnet_v2  import InceptionResNetV2   #Xception 
model = InceptionResNetV2 (weights='imagenet', include_top=False,input_shape = (img_width, img_height,3))
model.load_weights("C:/Users/Administrator/.keras/models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5")
# “小网络”结构
x = model.output
x = Flatten(input_shape=model.output_shape[1:])(x)
x = Dense(128, activation='relu', name='fc2',kernel_regularizer=regularizers.l2(0.01))(x)   
predictions = Dense(30, activation='softmax')(x)
# 两个网络整合
vgg_model = Model(input=model.input, output=predictions)
vgg_model.summary()

In [ ]:
### 冻结权重
for layer in vgg_model.layers[:438]:
    layer.trainable = False
###
vgg_model.compile(loss='categorical_crossentropy', optimizer='Adam', # sgd  Adadelta  RMSprop Adadelta
                  metrics=['accuracy'])


In [ ]:
from keras.callbacks import ModelCheckpoint 
# train the model
epochs = 100
nb_train_samples = 4886
nb_validation_samples = 119

batch_size = 16
#
checkpointer = ModelCheckpoint(filepath='vgg16.weights.best.hdf5', verbose=1, #Xcp
                               save_best_only=True)
vgg_model.fit_generator(train_generator, steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=valid_generator,
        validation_steps=nb_validation_samples // batch_size,
        callbacks=[checkpointer], verbose=1, shuffle=True)